In [ ]:
import vosk
import pyaudio
import json
import paho.mqtt.client as mqtt

# MQTT Configuration
mqtt_broker = "192.168.1.100"  # Replace with your laptop's IP
mqtt_topic = "home/devices"

# Initialize MQTT Client
mqtt_client = mqtt.Client()
mqtt_client.connect(mqtt_broker, 1883)

# Vosk Model Path (Ensure the model is extracted)
model = vosk.Model(lang="en-in")

# Define keywords
keywords = ["on", "off", "fan", "kitchen", "hall", "bedroom", "light", "terminate"]
rec = vosk.KaldiRecognizer(model, 16000, json.dumps(keywords))

# Initialize PyAudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=512)

# Function to send command via MQTT
def send_command(command):
    mqtt_client.publish(mqtt_topic, command)
    print(f"Sent command: {command}")

# Function to handle recognized text
def process_command(recognized_text):
    recognized_text = recognized_text.lower()
    
    if "terminate" in recognized_text:
        print("Termination keyword detected. Stopping...")
        return "terminate"
    
    # Map recognized speech to specific MQTT commands
    if "on hall" in recognized_text:
        send_command("on hall")
    elif "off hall" in recognized_text:
        send_command("off hall")
    elif "on kitchen" in recognized_text:
        send_command("on kitchen")
    elif "off kitchen" in recognized_text:
        send_command("off kitchen")
    elif "on bedroom" in recognized_text:
        send_command("on bedroom")
    elif "off bedroom" in recognized_text:
        send_command("off bedroom")
    elif "on fan" in recognized_text:
        send_command("on fan")
    elif "off fan" in recognized_text:
        send_command("off fan")
    
    return None

# Start Listening for Speech
print("Listening for commands. Say 'Terminate' to stop.")
try:
    while True:
        data = stream.read(512, exception_on_overflow=False)
        
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            recognized_text = result.get('text', "")
            
            if recognized_text:
                print(f"Recognized: {recognized_text}")
                if process_command(recognized_text) == "terminate":
                    break

finally:
    stream.stop_stream()
    stream.close()
    p.terminate()
    mqtt_client.disconnect()
    print("Disconnected from MQTT broker.")
